In [182]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import os
import parsers as p
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0101.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]

#df['time'] = pd.to_numeric(df['time'])
#print(np.any(df['time'][1:]-df['time'][:-1]<0))
#indices = np.where(df['time'][1:]-df['time'][:-1]<0)[0]+1
#df.iloc[indices]['time']=df['time'][indices]+1000
#df.iloc[265816-5:265816+5].head(10)

df['time'] = pd.to_datetime(df['time'], unit='ms')
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print(init_sz - pruned_sz)
print(time.time()-timea)

#df.head()

305
1.734907627105713


In [199]:
df = df.sample(frac=1).reset_index(drop=True)
df.sort_values(by=['time'],inplace=True)
df = df.reset_index(drop=True)

In [53]:
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()

In [200]:
unique, freq = np.unique(msg_id, return_counts = True)
hexes = np.array([p.MESSAGE_DICT[i][1] for i in unique])
print(np.array((freq,hexes)).T)

[['36366' 'MC1_status']
 ['35320' 'MC2_status']
 ['35719' 'MC3_status']
 ['35866' 'MC4_status']
 ['34911' 'MC1_temps']
 ['34759' 'MC2_temps']
 ['34847' 'MC3_temps']
 ['34755' 'MC4_temps']
 ['9194' 'MC1_energy']
 ['8694' 'MC2_energy']
 ['8899' 'MC3_energy']
 ['8992' 'MC4_energy']
 ['9237' 'MC1_setpoints_command']
 ['8805' 'MC2_setpoints_command']
 ['9279' 'MC3_setpoints_command']
 ['8892' 'MC4_setpoints_command']
 ['3430' 'MCU_status']
 ['6489' 'BMS_onboard_temperatures']
 ['54322' 'BMS_voltages']
 ['26135' 'BMS_detailed_voltages']
 ['12952' 'BMS_temperatures']
 ['8067' 'BMS_detailed_temperatures']
 ['3355' 'BMS_status']
 ['341' 'Dashboard_status']
 ['3397' 'EM_status']
 ['3409' 'EM_measurement']]


In [180]:
%%timeit -n 100
#timea=time.time()
a = p.parse_MC1_status(msg_id, msg_data)
b = p.parse_MC2_status(msg_id, msg_data)
c = p.parse_MC3_status(msg_id, msg_data)
d = p.parse_MC4_status(msg_id, msg_data)
#print((time.time()-timea))
#a.shape

10.4 ms ± 269 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [197]:
#%%timeit -n 20
timea=time.time()
points = 0
for i in p.MESSAGE_DICT.values():
    try:
        a = i[0](msg_id, msg_data)
        points += a.shape[0]
        print(i[1])
    except:
        continue
print(points)
print(points/msg_id.size)
print((time.time()-timea))

BMS_detailed_temperatures
BMS_detailed_voltages
BMS_onboard_temperatures
BMS_temperatures
BMS_voltages
MCU_load_cells
MC1_energy
MC2_energy
MC3_energy
MC4_energy
MC1_setpoints_command
MC2_setpoints_command
MC3_setpoints_command
MC4_setpoints_command
MC1_status
MC2_status
MC3_status
MC4_status
MC1_temps
MC2_temps
MC3_temps
MC4_temps
462500
0.9707576317291869
0.03072524070739746
